<a href="https://colab.research.google.com/github/buenoleonardo1/WebAnalytics/blob/main/API_Mercado_Libre_ARG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import requests
from google.oauth2 import service_account
from google.cloud import bigquery
from google.colab import files

# Fazer upload do arquivo de credenciais
uploaded = files.upload()

# Nome do arquivo carregado
filename = next(iter(uploaded))

# Configuração das credenciais do Google Cloud (BigQuery)
credentials = service_account.Credentials.from_service_account_file(filename)
client = bigquery.Client(credentials=credentials, project='api-mercado-livre-arg')

search_terms = ["chromecast", "Google Home", "Apple TV", "Amazon Fire TV", "Roku Ultra"]

# Lista para armazenar os detalhes dos itens
all_items = []

for term in search_terms:
    url = f"https://api.mercadolibre.com/sites/MLA/search?q={term}&limit=50"
    response = requests.get(url)
    data = response.json()

    for result in data['results']:
        item_id = result['id']
        item_url = f"https://api.mercadolibre.com/items/{item_id}"
        item_response = requests.get(item_url)
        item_data = item_response.json()

        # Adicione os detalhes relevantes do item à lista
        item_details = {
            'ID_da_categoria': str(item_data['category_id']),
            'ID_do_produto': str(item_data['id']),
            'Nome_do_Produto': str(item_data['title']),
            'Preco': float(item_data['price']),
            'Condicao': str(item_data['condition']),
            'Vendidos': int(item_data['sold_quantity']),
            'Estoque': int(item_data['available_quantity']),
            'Link': str(item_data['permalink']),
            # Adicione outras variáveis que você queira modelar
        }
        all_items.append(item_details)

# Crie um DataFrame a partir da lista de itens
df = pd.DataFrame(all_items)

# Envie o DataFrame para o BigQuery
table_id = 'api-mercado-livre-arg.mercado_livre_arg.sales_table_api'
df.to_gbq(destination_table=table_id, project_id=client.project, if_exists='replace')

print(f'Dados enviados para o BigQuery.')

# Exiba o DataFrame (head)
print(df.head())

df.to_csv('mlteste.csv', index=False)

Saving api-mercado-livre-arg-5b37ba497fd9.json to api-mercado-livre-arg-5b37ba497fd9.json


100%|██████████| 1/1 [00:00<00:00, 7989.15it/s]

Dados enviados para o BigQuery.
  ID_da_categoria  ID_do_produto  \
0       MLA352001  MLA1363188164   
1       MLA352001  MLA1376000264   
2       MLA352001  MLA1394845862   
3       MLA352001  MLA1382529583   
4       MLA352001  MLA1382770339   

                                     Nome_do_Produto    Preco Condicao  \
0  Chromecast Google Tv Hd 8gb 2gb Ram 60 Fps Wif...  48690.0      new   
1  Google Chromecast With Google Tv De Voz 4k 8gb...  62300.0      new   
2  Xiaomi Mi Tv Stick Mdz-24-aa - Negro - 8 Gb - ...  47799.0      new   
3  Google Chromecast With Google Tv De Voz 4k 8gb...  74999.0      new   
4  Tv Box Convertidor Smart 4k Netflix Youtube An...  29990.0      new   

   Vendidos  Estoque                                               Link  
0       500        1  https://articulo.mercadolibre.com.ar/MLA-13631...  
1       250        1  https://articulo.mercadolibre.com.ar/MLA-13760...  
2       500      200  https://articulo.mercadolibre.com.ar/MLA-13948...  
3         